In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
# This is needed to display the images.
%matplotlib inline

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util

In [4]:
# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

In [5]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())

In [6]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [7]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [8]:
from pykinect2 import PyKinectV2
from pykinect2.PyKinectV2 import *
from pykinect2 import PyKinectRuntime
from acquisitionKinect import AcquisitionKinect
from frame import Frame as Frame
import numpy as np
import cv2
import ctypes
import _ctypes
import sys
import face_recognition
import os

pykinect = PyKinectRuntime.PyKinectRuntime(PyKinectV2.FrameSourceTypes_Depth)
kinect = AcquisitionKinect()
f = Frame()

known_face_encodings = []
known_face_names = []

# Load sample pictures and learn how to recognize it.
dirname = 'face_recognition/knowns'
files = os.listdir(dirname)
for filename in files:
    name, ext = os.path.splitext(filename)
    if ext == '.jpg':
        known_face_names.append(name)
        pathname = os.path.join(dirname, filename)
        img = face_recognition.load_image_file(pathname)
        face_encoding = face_recognition.face_encodings(img)[0]
        known_face_encodings.append(face_encoding)

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    while True:
        
        # --- Getting frames and drawing
        kinect.get_frame(f)
        kinect.get_color_frame()
        image_np = kinect._frameRGB
        image_depth = kinect._frameDepthQuantized 
        image_np = cv2.cvtColor(image_np, cv2.COLOR_RGBA2RGB)
        image_np = image_np[ 200:1020, 350:1780]
        image_np = cv2.resize(image_np, (512,424))
        cv2.imshow("depth", image_depth)

        # ----- Face Recognition
        # Resize frame of video to 1/4 size for faster face recognition processing
        rgb_small_frame = cv2.resize(image_np, (0, 0), fx=0.25, fy=0.25)

        # Only process every other frame of video to save time
        if process_this_frame:
            # Find all the faces and face encodings in the current frame of video
            face_locations = face_recognition.face_locations(rgb_small_frame)
            face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

            face_names = []
            for face_encoding in face_encodings:
                # See if the face is a match for the known face(s)
                distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                min_value = min(distances)

                # tolerance: How much distance between faces to consider it a match. Lower is more strict.
                # 0.6 is typical best performance.
                name = "Unknown"
                if min_value < 0.9:
                    index = np.argmin(distances)
                    name = known_face_names[index]

                face_names.append(name)

        process_this_frame = not process_this_frame

        # Display the results
        for (top, right, bottom, left), name in zip(face_locations, face_names):
            # Scale back up face locations since the frame we detected in was scaled to 1/4 size
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4

            # Draw a box around the face
            cv2.rectangle(image_np, (left, top), (right, bottom), (0, 0, 255), 2)
            # Draw a label with a name below the face
            cv2.rectangle(image_np, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(image_np, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)


        # ----- Object Recognition
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')

        # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image_np, axis=0)
        # Actual detection.
        (boxes, scores, classes, num) = sess.run(
            [detection_boxes, detection_scores, detection_classes, num_detections],
            feed_dict={image_tensor: image_np_expanded})
        # Visualization of the results of a detection.
        vis_util.visualize_boxes_and_labels_on_image_array(
            image_np,
            np.squeeze(boxes),
            np.squeeze(classes).astype(np.int32),
            np.squeeze(scores),
            category_index,
            use_normalized_coordinates=True,
            line_thickness=8)

        # Get coordinates of detected boxes ; ymin, ymax, xmin, xmax
        coordinates = vis_util.return_coordinates(
                    image_np,
                    np.squeeze(boxes),
                    np.squeeze(classes).astype(np.int32),
                    np.squeeze(scores),
                    category_index,
                    use_normalized_coordinates=True,
                    line_thickness=8,
                    min_score_thresh=0.6)
        if coordinates is not None:

            # ----- Face Recognition
            print(face_names)
            count = 0
            for i in range(len(coordinates)):
                if coordinates[i][0] == 'person':
                    if count < len(face_names):
                        coordinates[i][0] = face_names[count]
                        count += 1

            #print(*coordinates)
            print(coordinates)
            xpoint_for_depth=[]
            ypoint_for_depth=[]

            print("object 개수:", len(coordinates))

            # Connect Color and Depth Info.
            frame = pykinect.get_last_depth_frame()
            #frameD = pykinect._depth_frame_data
            frameD = kinect.get_last_depth_frame()
            frame = frame.astype(np.uint8)
            frame = np.reshape(frame, (424, 512))
            frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)
            cv2.imshow("depth_frame", frame)

            # Take just one point : middle
            
            for i in range(len(coordinates)):

                ypoint_for_depth.append(int((coordinates[i][1]+coordinates[i][2])/2))
                xpoint_for_depth.append(int((coordinates[i][3]+coordinates[i][4])/2))

            print("ypoint_for_depth:", ypoint_for_depth)
            print("xpoint_for_depth:", xpoint_for_depth)
            

            ##output = cv2.bilateralFilter(output, 1, 150, 75)
            #cv2.imshow('KINECT Video Stream', frame)

            for i in range(len(coordinates)):
                #Pixel_Depth = frameD[((ypoint_for_depth[i] * 512) + xpoint_for_depth[i])]
                Pixel_Depth = image_depth[ypoint_for_depth[i]][xpoint_for_depth[i]]
                print("점 하나만 고려:",i+1, "번째 object의 depth value: ", Pixel_Depth)

            # Get average depth value considering all coordinate pairs. (List Version)
            '''Pixel_Depth_List = []
            Pixel_Depth_Sum = 0
            non_error_count = 0
            for i in range(len(coordinates)):
                for x in range(int(coordinates[i][3]), int(coordinates[i][4]+1)):
                    for y in range(int(coordinates[i][1]), int(coordinates[i][2]+1)):
                        Pixel_Depth_Sum += frameD[((y * 512) + x)]
                        if frameD[((y * 512) + x)] != 0:
                            non_error_count += 1

                Pixel_Depth_List.append(int(Pixel_Depth_Sum/non_error_count))

            for i in range(len(coordinates)):
                print("전체 면적 고려:", i+1, "번째 object의 depth value: ", Pixel_Depth_List[i])
            print("\n")'''

            # Get average depth value considering all coordinate pairs. (Dict Version)
            '''Pixel_Depth_Dict = {}
            Pixel_Depth_Sum = 0
            non_error_count = 0
            for i in range(len(coordinates)):

                # ----- Depth Recognition
                for x in range(int(coordinates[i][3]), int(coordinates[i][4]+1)):
                    for y in range(int(coordinates[i][1]), int(coordinates[i][2]+1)):
                        Pixel_Depth_Sum += frameD[((y * 512) + x)]
                        if frameD[((y * 512) + x)] != 0:
                            non_error_count += 1

                Pixel_Depth_Dict[coordinates[i][0]] = int(Pixel_Depth_Sum/non_error_count)

            # Compare each objects' depth. (Sorting: close -> far)
            sorted(Pixel_Depth_Dict.items(), key=lambda t : t[1])
            print(Pixel_Depth_Dict)
            print("\n")'''

            # VideoFileOutput.write(image_np)
            cv2.imshow('KINECT Color', image_np)
            cv2.waitKey(1)


        key = cv2.waitKey(1)
        if key == 27: break

[]
[['person', 112, 355, 58, 164, 95.96155881881714], ['surfboard', 297, 424, 64, 500, 65.38621187210083], ['person', 117, 322, 251, 465, 61.543381214141846]]
object 개수: 3
ypoint_for_depth: [233, 360, 219]
xpoint_for_depth: [111, 282, 358]
점 하나만 고려: 1 번째 object의 depth value:  155
점 하나만 고려: 2 번째 object의 depth value:  194
점 하나만 고려: 3 번째 object의 depth value:  53
[]
[['person', 129, 347, 145, 223, 97.14494943618774], ['person', 151, 356, 63, 175, 75.0015377998352]]
object 개수: 2
ypoint_for_depth: [238, 253]
xpoint_for_depth: [184, 119]
점 하나만 고려: 1 번째 object의 depth value:  101
점 하나만 고려: 2 번째 object의 depth value:  91
[]
[['person', 116, 357, 130, 218, 97.32073545455933]]
object 개수: 1
ypoint_for_depth: [236]
xpoint_for_depth: [174]
점 하나만 고려: 1 번째 object의 depth value:  104
[]
[['person', 116, 352, 118, 213, 95.645010471344]]
object 개수: 1
ypoint_for_depth: [234]
xpoint_for_depth: [165]
점 하나만 고려: 1 번째 object의 depth value:  119
[]
[['person', 112, 350, 110, 210, 94.91596817970276]]
object 개수: 1
yp

[]
[['person', 44, 363, 58, 188, 91.63768291473389]]
object 개수: 1
ypoint_for_depth: [203]
xpoint_for_depth: [123]
점 하나만 고려: 1 번째 object의 depth value:  75
[]
[['person', 43, 360, 59, 179, 84.46287512779236], ['person', 4, 348, 272, 511, 63.05287480354309]]
object 개수: 2
ypoint_for_depth: [201, 176]
xpoint_for_depth: [119, 391]
점 하나만 고려: 1 번째 object의 depth value:  69
점 하나만 고려: 2 번째 object의 depth value:  18
[]
[['person', 41, 352, 62, 181, 86.39265298843384], ['person', 3, 346, 270, 511, 67.36468076705933]]
object 개수: 2
ypoint_for_depth: [196, 174]
xpoint_for_depth: [121, 390]
점 하나만 고려: 1 번째 object의 depth value:  66
점 하나만 고려: 2 번째 object의 depth value:  17
[]
[['person', 40, 354, 62, 183, 79.47386503219604], ['person', 5, 346, 269, 511, 67.95357465744019]]
object 개수: 2
ypoint_for_depth: [197, 175]
xpoint_for_depth: [122, 390]
점 하나만 고려: 1 번째 object의 depth value:  63
점 하나만 고려: 2 번째 object의 depth value:  15
[]
[['person', 20, 357, 61, 187, 89.55914378166199], ['person', 5, 350, 268, 511, 68.99

[]
[['person', 89, 359, 55, 181, 95.27048468589783], ['surfboard', 296, 424, 61, 501, 67.23008155822754], ['person', 0, 347, 263, 512, 64.98144865036011]]
object 개수: 3
ypoint_for_depth: [224, 360, 173]
xpoint_for_depth: [118, 281, 387]
점 하나만 고려: 1 번째 object의 depth value:  122
점 하나만 고려: 2 번째 object의 depth value:  194
점 하나만 고려: 3 번째 object의 depth value:  19
[]
[['person', 89, 359, 55, 181, 94.70427632331848], ['surfboard', 295, 424, 61, 502, 64.9751365184784], ['person', 1, 361, 262, 508, 63.45624327659607]]
object 개수: 3
ypoint_for_depth: [224, 359, 181]
xpoint_for_depth: [118, 281, 385]
점 하나만 고려: 1 번째 object의 depth value:  122
점 하나만 고려: 2 번째 object의 depth value:  194
점 하나만 고려: 3 번째 object의 depth value:  19
[]
[['person', 90, 358, 55, 179, 94.74951028823853], ['surfboard', 295, 424, 63, 501, 67.15222597122192], ['person', 0, 344, 265, 512, 67.05206036567688]]
object 개수: 3
ypoint_for_depth: [224, 359, 172]
xpoint_for_depth: [117, 282, 388]
점 하나만 고려: 1 번째 object의 depth value:  122
점 하나만 고려

[]
[['person', 95, 361, 59, 177, 90.83112478256226], ['surfboard', 298, 424, 61, 503, 72.19061851501465], ['person', 1, 355, 269, 509, 64.19376730918884]]
object 개수: 3
ypoint_for_depth: [228, 361, 178]
xpoint_for_depth: [118, 282, 389]
점 하나만 고려: 1 번째 object의 depth value:  123
점 하나만 고려: 2 번째 object의 depth value:  194
점 하나만 고려: 3 번째 object의 depth value:  20
[]
[['person', 95, 371, 58, 176, 90.32750129699707], ['surfboard', 297, 424, 63, 503, 68.2316780090332], ['person', 1, 352, 290, 510, 62.261176109313965]]
object 개수: 3
ypoint_for_depth: [233, 360, 176]
xpoint_for_depth: [117, 283, 400]
점 하나만 고려: 1 번째 object의 depth value:  122
점 하나만 고려: 2 번째 object의 depth value:  194
점 하나만 고려: 3 번째 object의 depth value:  14
[]
[['person', 95, 361, 59, 178, 90.19426107406616], ['surfboard', 296, 424, 63, 502, 68.91762614250183], ['person', 2, 356, 303, 509, 61.570435762405396]]
object 개수: 3
ypoint_for_depth: [228, 360, 179]
xpoint_for_depth: [118, 282, 406]
점 하나만 고려: 1 번째 object의 depth value:  123
점 하나만 

[]
[['person', 10, 352, 274, 509, 66.52075052261353], ['surfboard', 296, 423, 67, 502, 61.47066354751587]]
object 개수: 2
ypoint_for_depth: [181, 359]
xpoint_for_depth: [391, 284]
점 하나만 고려: 1 번째 object의 depth value:  32
점 하나만 고려: 2 번째 object의 depth value:  194
[]
[['person', 93, 362, 54, 168, 80.70250749588013], ['person', 2, 347, 278, 510, 74.25116896629333]]
object 개수: 2
ypoint_for_depth: [227, 174]
xpoint_for_depth: [111, 394]
점 하나만 고려: 1 번째 object의 depth value:  119
점 하나만 고려: 2 번째 object의 depth value:  29
[]
[['person', 94, 371, 58, 182, 86.67763471603394], ['person', 0, 346, 280, 509, 69.87816095352173]]
object 개수: 2
ypoint_for_depth: [232, 173]
xpoint_for_depth: [120, 394]
점 하나만 고려: 1 번째 object의 depth value:  61
점 하나만 고려: 2 번째 object의 depth value:  28
[]
[['person', 96, 360, 59, 180, 92.29093790054321], ['person', 0, 344, 281, 509, 69.03632879257202]]
object 개수: 2
ypoint_for_depth: [228, 172]
xpoint_for_depth: [119, 395]
점 하나만 고려: 1 번째 object의 depth value:  67
점 하나만 고려: 2 번째 object

...

[]
[['person', 0, 370, 51, 219, 64.56684470176697]]
object 개수: 1
ypoint_for_depth: [185]
xpoint_for_depth: [135]
점 하나만 고려: 1 번째 object의 depth value:  43
[]
[['chair', 219, 360, 282, 353, 66.4632499217987]]
object 개수: 1
ypoint_for_depth: [289]
xpoint_for_depth: [317]
점 하나만 고려: 1 번째 object의 depth value:  57
[]
[['chair', 218, 359, 282, 351, 67.6823377609253], ['person', 2, 368, 49, 221, 64.23218250274658]]
object 개수: 2
ypoint_for_depth: [288, 185]
xpoint_for_depth: [316, 135]
점 하나만 고려: 1 번째 object의 depth value:  55
점 하나만 고려: 2 번째 object의 depth value:  29
[]
[['chair', 218, 359, 282, 351, 67.05397367477417], ['person', 0, 360, 51, 217, 64.64411616325378]]
object 개수: 2
ypoint_for_depth: [288, 180]
xpoint_for_depth: [316, 134]
점 하나만 고려: 1 번째 object의 depth value:  54
점 하나만 고려: 2 번째 object의 depth value:  25
[]
[['chair', 219, 359, 281, 351, 66.21805429458618]]
object 개수: 1
ypoint_for_depth: [289]
xpoint_for_depth: [316]
점 하나만 고려: 1 번째 object의 depth value:  53
[]
[['chair', 218, 359, 279, 349,

[]
[['person', 6, 372, 56, 204, 82.032710313797], ['chair', 225, 360, 281, 340, 60.40751338005066]]
object 개수: 2
ypoint_for_depth: [189, 292]
xpoint_for_depth: [130, 310]
점 하나만 고려: 1 번째 object의 depth value:  53
점 하나만 고려: 2 번째 object의 depth value:  62
[]
[['person', 6, 368, 57, 202, 87.19406127929688], ['chair', 225, 360, 281, 341, 62.382519245147705]]
object 개수: 2
ypoint_for_depth: [187, 292]
xpoint_for_depth: [129, 311]
점 하나만 고려: 1 번째 object의 depth value:  53
점 하나만 고려: 2 번째 object의 depth value:  63
[]
[['person', 5, 366, 58, 207, 80.3476333618164], ['chair', 224, 361, 280, 339, 64.56277966499329]]
object 개수: 2
ypoint_for_depth: [185, 292]
xpoint_for_depth: [132, 309]
점 하나만 고려: 1 번째 object의 depth value:  53
점 하나만 고려: 2 번째 object의 depth value:  61
[]
[['person', 5, 367, 57, 207, 78.73397469520569], ['chair', 225, 361, 280, 339, 62.775981426239014]]
object 개수: 2
ypoint_for_depth: [186, 293]
xpoint_for_depth: [132, 309]
점 하나만 고려: 1 번째 object의 depth value:  53
점 하나만 고려: 2 번째 object의 depth 

[]
[['person', 141, 355, 54, 153, 87.34139204025269], ['chair', 227, 362, 276, 342, 60.2481484413147]]
object 개수: 2
ypoint_for_depth: [248, 294]
xpoint_for_depth: [103, 309]
점 하나만 고려: 1 번째 object의 depth value:  142
점 하나만 고려: 2 번째 object의 depth value:  60
[]
[['person', 141, 355, 54, 152, 85.97822189331055]]
object 개수: 1
ypoint_for_depth: [248]
xpoint_for_depth: [103]
점 하나만 고려: 1 번째 object의 depth value:  143
[]
[['person', 141, 357, 56, 146, 87.81808614730835], ['chair', 227, 362, 275, 340, 64.54956531524658]]
object 개수: 2
ypoint_for_depth: [249, 294]
xpoint_for_depth: [101, 307]
점 하나만 고려: 1 번째 object의 depth value:  145
점 하나만 고려: 2 번째 object의 depth value:  58
[]
[['person', 142, 357, 57, 146, 76.99973583221436], ['chair', 226, 362, 275, 340, 63.783228397369385]]
object 개수: 2
ypoint_for_depth: [249, 294]
xpoint_for_depth: [101, 307]
점 하나만 고려: 1 번째 object의 depth value:  147
점 하나만 고려: 2 번째 object의 depth value:  58
[]
[['person', 142, 358, 55, 154, 68.80108118057251], ['chair', 226, 362, 27

[]
[['chair', 218, 359, 280, 336, 72.10837006568909]]
object 개수: 1
ypoint_for_depth: [288]
xpoint_for_depth: [308]
점 하나만 고려: 1 번째 object의 depth value:  59
[]
[['chair', 218, 359, 280, 338, 72.95216917991638]]
object 개수: 1
ypoint_for_depth: [288]
xpoint_for_depth: [309]
점 하나만 고려: 1 번째 object의 depth value:  62
[]
[['chair', 224, 361, 281, 343, 68.87321472167969]]
object 개수: 1
ypoint_for_depth: [292]
xpoint_for_depth: [312]
점 하나만 고려: 1 번째 object의 depth value:  63
[]
[['person', 0, 377, 28, 261, 60.01219153404236]]
object 개수: 1
ypoint_for_depth: [188]
xpoint_for_depth: [144]
점 하나만 고려: 1 번째 object의 depth value:  28
[]
[]
object 개수: 0
ypoint_for_depth: []
xpoint_for_depth: []
[]
[]
object 개수: 0
ypoint_for_depth: []
xpoint_for_depth: []
[]
[]
object 개수: 0
ypoint_for_depth: []
xpoint_for_depth: []
[]
[['person', 5, 369, 58, 480, 66.10857844352722]]
object 개수: 1
ypoint_for_depth: [187]
xpoint_for_depth: [269]
점 하나만 고려: 1 번째 object의 depth value:  194
[]
[]
object 개수: 0
ypoint_for_depth: []
xpoin

ValueError: math domain error